In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
# #read in the data
# mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
# mid.drop([0,1], inplace=True)
# print(mid.describe())
# #bring the time data into a new df
# time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
# print(time.describe())
# #add elapsed time columns
# def elapsed_time(df):
#     itr = 0
#     counter = len(df.columns)
#     masses = [2,4,5.5,12,15,16,18,28,32,40,44]
#     for i in df:
#         timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
#         runtime = (timestamp-timestamp[2]).dt.total_seconds()
#         colname = "Elapsed time, mass {}".format(masses[itr]) 
#         df.insert(counter, colname, runtime)
#         itr += 1
#         counter += 1
#     return(df)
# elapsed_time(time)

# #time["Elapsed time, mass 40"].describe()

In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,11/01/2023 14:55:06,8.0,-0.063035,2.490000e-09,9.750000e-10,4.996000e-09,6.553000e-07,0.000526,0.0,-106.060,58.249,NaN,NaN
2,NaN,11/01/2023 14:55:13,15.0,-0.062886,2.480000e-09,9.730000e-10,4.996000e-09,6.553000e-07,0.000533,0.0,-105.836,58.137,NaN,NaN
3,NaN,11/01/2023 14:55:20,22.0,-0.062147,2.490000e-09,9.730000e-10,4.996000e-09,6.553000e-07,0.000513,0.0,-105.723,58.361,NaN,NaN
4,NaN,11/01/2023 14:55:27,29.0,-0.058472,2.490000e-09,9.760000e-10,4.996000e-09,6.561000e-07,0.000530,0.0,-105.836,58.361,NaN,NaN
5,NaN,11/01/2023 14:55:34,36.0,-0.061628,2.480000e-09,9.730000e-10,4.996000e-09,6.585000e-07,0.000529,0.0,-105.948,58.361,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])
<
#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



SyntaxError: invalid syntax (3764689485.py, line 9)

### Data processing

In [6]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [ ]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="emission on"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
x=pd.Timestamp(2022, 12, 13, 17,38,53)
y=pd.Timestamp(2022, 12, 13, 17, 45, 21)

#then find the time value closest to it in the MID scan file 
print("\n","Result 3:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 40.0_value"])

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck

 
#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][45:107]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:45]/h2_sens
base_h2.describe()

### Plotting

In [ ]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(200, 2.5e-11, 'No emission', fontdict = font)
plt.text(525, 2.9e-11, '\u2193 Initial desorption peak', fontdict = font)
plt.text(1100, 2.6e-11, 'Grid transparent', fontdict = font)
plt.text(1670, 1.5e-11, 'Grid repelling', fontdict = font)
plt.text(2005, 4.46e-11, '\u2193 EGA rotated', fontdict = font)
plt.text(2306, 2.6e-11, 'Grid transparent', fontdict = font)
plt.yscale('log')

In [ ]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")


plt.yscale('log')


In [ ]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [8]:
## Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")
plt.yscale('linear')

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.

Due to the EGA being fully retracted, it is safe to assume that the desorption takes place at approximately room temperature

In [ ]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 293
print(T)
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current
#ignore the inrush current values
hv["em"].where(hv["em"] <= 1.5e-6, np.nan, inplace=True)

I_em = np.nanmean(hv["em"].nlargest(30))
print(I_em)
#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000


In [ ]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)